In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import lightgbm as lgb
import logging
import pickle
import gc

In [2]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [3]:
base_path = './data'
feature_path = './feature'

In [4]:
# train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
# train.columns = ['qid', 'uid', 'dt', 'label']

# del train['dt']
# logging.info("train %s", train.shape)

# test = pd.read_csv(f'{base_path}/invite_info_evaluate_0926.txt', sep='\t', header=None)
# test.columns = ['qid', 'uid', 'dt']
# sub = test.copy()
# sub_size = len(sub)

# del test['dt']
# logging.info("test %s", test.shape)

[2019-12-14 06:51:43,107] INFO in <ipython-input-4-c899d16ad23a>: train (9489162, 3)
[2019-12-14 06:51:44,792] INFO in <ipython-input-4-c899d16ad23a>: test (1141683, 2)


In [5]:
# len_train = len(train)
# len_test = len(test)

In [6]:
# qid feature
# t1 = pd.read_pickle(f'{feature_path}/train_qid_feature.pkl')
# t1 = t1.reset_index(drop= True)
# train = train.reset_index(drop= True)
# train = pd.concat([train, t1], axis=1)
# logging.info("train %s", train.shape)

# t1 = pd.read_pickle(f'{feature_path}/test_qid_feature.pkl')
# t1 = t1.reset_index(drop= True)
# test = test.reset_index(drop= True)
# test = pd.concat([test, t1], axis=1)
# logging.info("test %s", test.shape)

[2019-12-14 06:52:18,489] INFO in <ipython-input-6-277cc9743066>: train (9489162, 170)
[2019-12-14 06:52:22,568] INFO in <ipython-input-6-277cc9743066>: test (1141683, 169)


In [7]:
# 加载 history 
# t1 = pickle.load(open(f'{feature_path}/history_feature.pkl', 'rb'))

# train = pd.concat([train, t1[:len_train]], axis=1)
# logging.info("train %s", train.shape)

# test = pd.concat([test, t1[len_train:].reset_index(drop=True)], axis=1)
# logging.info("test %s", test.shape)

[2019-12-14 06:52:46,162] INFO in <ipython-input-7-6525bd882acf>: train (9489162, 196)
[2019-12-14 06:52:48,742] INFO in <ipython-input-7-6525bd882acf>: test (1141683, 195)


In [8]:
# 加载 history 1
# t1 = pickle.load(open(f'{feature_path}/history_feature1.pkl', 'rb'))

# train = pd.concat([train, t1[:len_train]], axis=1)
# logging.info("train %s", train.shape)

# test = pd.concat([test, t1[len_train:].reset_index(drop=True)], axis=1)
# logging.info("test %s", test.shape)

[2019-12-14 06:53:15,141] INFO in <ipython-input-8-e35bf51534b0>: train (9489162, 222)
[2019-12-14 06:53:18,117] INFO in <ipython-input-8-e35bf51534b0>: test (1141683, 221)


In [9]:
# 加载 history 2
# t1 = pickle.load(open(f'{feature_path}/history_feature2.pkl', 'rb'))

# train = pd.concat([train, t1[:len_train]], axis=1)
# logging.info("train %s", train.shape)

# test = pd.concat([test, t1[len_train:].reset_index(drop=True)], axis=1)
# logging.info("test %s", test.shape)

[2019-12-14 06:53:48,047] INFO in <ipython-input-9-2ebbf6b9bccc>: train (9489162, 248)
[2019-12-14 06:53:51,407] INFO in <ipython-input-9-2ebbf6b9bccc>: test (1141683, 247)


In [13]:
# 加载 ans kfold feature
# all_col = ['day', 'hour', 'q_inv_kfold_mean', 'q_inv_kfold_sum', 'q_inv_kfold_std', 'q_inv_kfold_count', 
#            'u_inv_kfold_mean', 'u_inv_kfold_sum', 'u_inv_kfold_std', 'u_inv_kfold_count', 'q_ans_kfold_count',
#            'u_ans_kfold_count', 'q_is_good_sum', 'q_is_good_max', 'q_is_good_mean', 'u_is_good_sum',
#            'u_is_good_max', 'u_is_good_mean', 'q_is_rec_sum', 'q_is_rec_max', 'q_is_rec_mean', 'u_is_rec_sum',
#            'u_is_rec_max', 'u_is_rec_mean', 'q_is_dest_sum', 'q_is_dest_max', 'q_is_dest_mean', 
#            'u_is_dest_sum', 'u_is_dest_max', 'u_is_dest_mean', 'q_has_img_sum', 'q_has_img_max', 
#            'q_has_img_mean', 'u_has_img_sum', 'u_has_img_max', 'u_has_img_mean', 'q_has_video_sum', 
#            'q_has_video_max', 'q_has_video_mean', 'u_has_video_sum', 'u_has_video_max', 'u_has_video_mean',
#            'q_word_count_sum', 'q_word_count_max', 'q_word_count_mean', 'u_word_count_sum', 'u_word_count_max',
#            'u_word_count_mean', 'q_reci_cheer_sum', 'q_reci_cheer_max', 'q_reci_cheer_mean', 'u_reci_cheer_sum',
#            'u_reci_cheer_max', 'u_reci_cheer_mean', 'q_reci_uncheer_sum', 'q_reci_uncheer_max', 
#            'q_reci_uncheer_mean', 'u_reci_uncheer_sum', 'u_reci_uncheer_max', 'u_reci_uncheer_mean', 
#            'q_reci_comment_sum', 'q_reci_comment_max', 'q_reci_comment_mean', 'u_reci_comment_sum', 
#            'u_reci_comment_max', 'u_reci_comment_mean', 'q_reci_mark_sum', 'q_reci_mark_max', 
#            'q_reci_mark_mean', 'u_reci_mark_sum', 'u_reci_mark_max', 'u_reci_mark_mean', 'q_reci_tks_sum',
#            'q_reci_tks_max', 'q_reci_tks_mean', 'u_reci_tks_sum', 'u_reci_tks_max', 'u_reci_tks_mean',
#            'q_reci_xxx_sum', 'q_reci_xxx_max', 'q_reci_xxx_mean', 'u_reci_xxx_sum', 'u_reci_xxx_max', 
#            'u_reci_xxx_mean', 'q_reci_no_help_sum', 'q_reci_no_help_max', 'q_reci_no_help_mean', 
#            'u_reci_no_help_sum', 'u_reci_no_help_max', 'u_reci_no_help_mean', 'q_reci_dis_sum', 
#            'q_reci_dis_max', 'q_reci_dis_mean', 'u_reci_dis_sum', 'u_reci_dis_max', 'u_reci_dis_mean', 
#            'q_diff_qa_days_sum', 'q_diff_qa_days_max', 'q_diff_qa_days_mean', 'u_diff_qa_days_sum', 
#            'u_diff_qa_days_max', 'u_diff_qa_days_mean']
# drop_col = ['u_is_rec_mean', 'u_reci_uncheer_mean', 'q_is_dest_sum', 'u_reci_uncheer_sum', 'u_is_rec_max', 
#              'u_is_dest_mean','q_reci_uncheer_mean', 'q_reci_uncheer_sum', 'u_is_dest_sum', 'q_is_dest_max',
#              'q_reci_uncheer_max', 'u_reci_tks_max', 'q_reci_mark_max','u_reci_dis_max', 'q_has_video_mean',
#              'q_reci_no_help_mean', 'count_u_topic', 'u_has_video_mean', 'q_reci_dis_sum', 'q_reci_mark_sum',
#              'q_reci_tks_sum','q_reci_tks_max','q_reci_dis_max','u_reci_mark_max','q_is_good_mean',
#              'q_reci_no_help_sum', 'q_reci_xxx_max', 'u_reci_xxx_max','u_reci_no_help_sum','u_reci_xxx_sum',
#               'u_is_good_mean','q_reci_no_help_max','u_has_img_max','u_is_good_sum','u_reci_no_help_max',
#               'u_has_video_sum','uf_b5','q_reci_xxx_sum','q_is_good_sum','q_has_img_max','q_has_video_sum',
#               'q_has_video_max','u_has_video_max','q_is_good_max','q_is_rec_max','u_is_good_max',
#               'q_is_dest_mean','u_reci_uncheer_max','uf_c5_count','u_is_dest_max','q_is_rec_mean',
#               'q_is_rec_sum','u_is_rec_sum', 'q_reci_xxx_mean','u_reci_xxx_mean','u_reci_comment_max',
#               'q_reci_comment_sum','u_reci_cheer_max','u_reci_dis_sum','u_reci_tks_sum','q_has_img_sum',
#               'q_reci_comment_max','q_reci_cheer_max','u_reci_no_help_mean','u_has_img_sum','u_reci_mark_sum']
# use_col = list(set(all_col) - set(drop_col))

# t1 = pd.read_csv(f'{feature_path}/train_kfold_feature.txt', sep='\t', usecols=use_col)
# train = pd.concat([train, t1], axis=1)
# logging.info("train %s", train.shape)


# t1 = pd.read_csv(f'{feature_path}/test_kfold_feature.txt', sep='\t', usecols=use_col)
# test = pd.concat([test, t1], axis=1)
# logging.info("test %s", test.shape)

[2019-12-14 06:56:42,177] INFO in <ipython-input-13-df91efe901e4>: train (9489162, 287)
[2019-12-14 06:56:52,064] INFO in <ipython-input-13-df91efe901e4>: test (1141683, 286)


In [16]:
# t1 = []
# n_dup = 0
# for c in train.columns:
#     if c not in t1:
#         t1.append(c)
#     else:
#         t1.append(c+'_2')
#         print(c, c+'_2')
#         n_dup += 1
# print('dup num: ', n_dup)
# print('=================')

# t2 = []
# n_dup = 0
# for c in test.columns:
#     if c not in t2:
#         t2.append(c)
#     else:
#         t2.append(c+'_2')
#         print(c, c+'_2')
#         n_dup += 1
# print('dup num: ', n_dup)
# #         print(c+'_2')
# print(len(t1))
# print(len(t2))

# train.columns = t1
# test.columns = t2
# del train['day_2'], test['day_2']
# gc.collect()

dup num:  0
uid_history_inv_count uid_history_inv_count_2
uid_history_label_mean uid_history_label_mean_2
qid_history_inv_count qid_history_inv_count_2
qid_history_label_mean qid_history_label_mean_2
qid_history_uf_b1_mean qid_history_uf_b1_mean_2
qid_history_uf_b1_std qid_history_uf_b1_std_2
qid_history_uf_b2_mean qid_history_uf_b2_mean_2
qid_history_uf_b2_std qid_history_uf_b2_std_2
qid_history_uf_b3_mean qid_history_uf_b3_mean_2
qid_history_uf_b3_std qid_history_uf_b3_std_2
qid_history_uf_b4_mean qid_history_uf_b4_mean_2
qid_history_uf_b4_std qid_history_uf_b4_std_2
qid_history_uf_b5_mean qid_history_uf_b5_mean_2
qid_history_uf_b5_std qid_history_uf_b5_std_2
qid_history_uf_c1_mean qid_history_uf_c1_mean_2
qid_history_uf_c1_std qid_history_uf_c1_std_2
qid_history_uf_c2_mean qid_history_uf_c2_mean_2
qid_history_uf_c2_std qid_history_uf_c2_std_2
qid_history_uf_c3_mean qid_history_uf_c3_mean_2
qid_history_uf_c3_std qid_history_uf_c3_std_2
qid_history_uf_c4_mean qid_history_uf_c4_mean_2


In [22]:
# train['week'] = train['day']%7
# test['week'] = test['day']%7

In [23]:
# 加载 invete feature 1
# t1 = pd.read_csv(f'{feature_path}/train_invite_feature.txt', sep='\t')
# train = pd.concat([train, t1], axis=1)
# logging.info("train %s", train.shape)

# t1 = pd.read_csv(f'{feature_path}/test_invite_feature.txt', sep='\t')
# test = pd.concat([test, t1], axis=1)
# logging.info("test %s", test.shape)

[2019-12-14 07:03:35,685] INFO in <ipython-input-23-25a44eae3c1c>: train (9489162, 305)
[2019-12-14 07:03:42,273] INFO in <ipython-input-23-25a44eae3c1c>: test (1141683, 304)


In [24]:
# 加载 invete feature 2
# t1 = pd.read_csv(f'{feature_path}/train_invite_feature_2.txt', sep='\t')
# train = pd.concat([train, t1], axis=1)
# logging.info("train %s", train.shape)

# t1 = pd.read_csv(f'{feature_path}/test_invite_feature_2.txt', sep='\t')
# test = pd.concat([test, t1], axis=1)
# logging.info("test %s", test.shape)

[2019-12-14 07:04:50,430] INFO in <ipython-input-24-898ae5923687>: train (9489162, 333)
[2019-12-14 07:04:58,287] INFO in <ipython-input-24-898ae5923687>: test (1141683, 332)


In [25]:
# 加载 kfold topic feature, QU
# t1 = pd.read_csv(f'{feature_path}/train_kfold_topic_feature.txt', sep='\t')
# train = pd.concat([train, t1], axis=1)
# logging.info("train %s", train.shape)

# t1 = pd.read_csv(f'{feature_path}/test_kfold_topic_feature.txt', sep='\t')
# test = pd.concat([test, t1], axis=1)
# logging.info("test %s", test.shape)

[2019-12-14 07:06:27,612] INFO in <ipython-input-25-dff71a8a2332>: train (9489162, 357)
[2019-12-14 07:06:36,460] INFO in <ipython-input-25-dff71a8a2332>: test (1141683, 356)


In [26]:
# 加载 user kfold topic feature，UU
# t1 = pd.read_csv(f'{feature_path}/train_kfold_ut_feature.txt', sep='\t')
# train = pd.concat([train, t1], axis=1)
# logging.info("train %s", train.shape)

# t1 = pd.read_csv(f'{feature_path}/test_kfold_ut_feature.txt', sep='\t')
# test = pd.concat([test, t1], axis=1)
# logging.info("test %s", test.shape)

[2019-12-14 07:08:26,674] INFO in <ipython-input-26-35a09705a8ca>: train (9489162, 373)
[2019-12-14 07:08:34,000] INFO in <ipython-input-26-35a09705a8ca>: test (1141683, 372)


In [31]:
# 压缩数据

# t = train.dtypes
# for x in t[t == 'int64'].index:
#     train[x] = train[x].astype('int32')
# for x in t[t == 'float64'].index:
#     train[x] = train[x].astype('float32')

# t = test.dtypes
# for x in t[t == 'int64'].index:
#     test[x] = test[x].astype('int32')
# for x in t[t == 'float64'].index:
#     test[x] = test[x].astype('float32')

# pickle.dump(train, open(f'{feature_path}/train_373.pkl', 'wb'))
# pickle.dump(test, open(f'{feature_path}/test_373.pkl', 'wb'))

In [ ]:
train = pickle.load(open(f'{feature_path}/train_373.pkl', 'rb'))
test = pickle.load(open(f'{feature_path}/test_373.pkl', 'rb'))

In [34]:
# 加载用户
# user = pd.read_csv(f'{base_path}/member_info_0926.txt', header=None, sep='\t')
# user.columns = ['uid', 'gender', 'freq', 'uf_b1', 'uf_b2','uf_b3', 'uf_b4', 'uf_b5', 
#                 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5',  'score', 'follow_topic', 'inter_topic']

# del user['follow_topic'], user['inter_topic']
# logging.info("user %s", user.shape)

# unq = user.nunique()
# logging.info("user unq %s", unq)

# for x in unq[unq == 1].index:
#     del user[x]
#     logging.info('del unq==1 %s', x)

# t = user.dtypes
# cats = [x for x in t[t == 'object'].index if x not in ['follow_topic', 'inter_topic', 'uid']]
# logging.info("user cat %s", cats)

# for d in cats:
#     lb = LabelEncoder()
#     user[d] = lb.fit_transform(user[d])
#     logging.info('encode %s', d)
    
# logging.info('encoding qid...')    
# q_lb = LabelEncoder()
# q_lb.fit(list(train['qid'].astype(str).values) + list(test['qid'].astype(str).values))
# train['qid_enc'] = q_lb.transform(train['qid'])
# test['qid_enc'] = q_lb.transform(test['qid'])
# logging.info('add qid_enc')

# logging.info('encoding uid...')
# u_lb = LabelEncoder()
# u_lb.fit(user['uid'])
# train['uid_enc'] = u_lb.transform(train['uid'])
# test['uid_enc'] = u_lb.transform(test['uid'])
# logging.info('add uid_enc')

# # merge user
# train = pd.merge(train, user, on='uid', how='left')
# test = pd.merge(test, user, on='uid', how='left')
# logging.info("train shape %s, test shape %s", train.shape, test.shape)

# del user
# gc.collect()

[2019-12-14 07:35:51,491] INFO in <ipython-input-34-a43802a9c4ae>: user (1931654, 14)
[2019-12-14 07:35:55,675] INFO in <ipython-input-34-a43802a9c4ae>: user unq uid       1931654
gender          3
freq            5
uf_b1           2
uf_b2           2
uf_b3           2
uf_b4           2
uf_b5           2
uf_c1        2561
uf_c2         291
uf_c3         428
uf_c4        1556
uf_c5           2
score         732
dtype: int64
[2019-12-14 07:35:55,681] INFO in <ipython-input-34-a43802a9c4ae>: user cat ['gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
[2019-12-14 07:35:56,732] INFO in <ipython-input-34-a43802a9c4ae>: encode gender
[2019-12-14 07:35:57,738] INFO in <ipython-input-34-a43802a9c4ae>: encode freq
[2019-12-14 07:35:58,679] INFO in <ipython-input-34-a43802a9c4ae>: encode uf_c1
[2019-12-14 07:35:59,617] INFO in <ipython-input-34-a43802a9c4ae>: encode uf_c2
[2019-12-14 07:36:00,492] INFO in <ipython-input-34-a43802a9c4ae>: encode uf_c3
[2019-12-14 07:36:01,333] INFO in

262

In [ ]:
# 用户问题等信息 encoding
t1 = pd.read_pickle(f'{feature_path}/train_member_basic_feature.pkl')
print(t1.keys())
t1 = t1.reset_index(drop= True)
train = train[[f for f in train.keys() if f not in t1.keys()]]
train = train.reset_index(drop= True)
train = pd.concat([train, t1], axis=1)

t1 = pd.read_pickle(f'{feature_path}/test_member_basic_feature.pkl')
t1 = t1.reset_index(drop= True)
test = test[[f for f in test.keys() if f not in t1.keys()]]
test = test.reset_index(drop= True)
test = pd.concat([test, t1], axis=1)

In [35]:
data = pd.concat((train, test), axis=0, sort=True)
len_train = len(train)

del train
gc.collect()

0

In [36]:
t1 = pd.read_csv(f'{feature_path}/count_features.txt', sep='\t')
print(len(t1), len(data))
data = data[[f for f in data.keys() if f not in t1.keys()]]
t1 = t1.reset_index(drop= True)
data = data.reset_index(drop= True)
data = pd.concat([data, t1], axis=1)

10630845 10630845


In [ ]:
del t1
gc.collect()

In [ ]:
# count 特征
# count_feat = ['uid_enc', 'qid_enc', 'gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
# for feat in count_feat:
#     logging.info('counting %s', feat)
#     col_name = '{}_count'.format(feat)
#     data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
#     data.loc[data[col_name] < 2, feat] = -1
#     data[feat] += 1
#     data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
#     data[col_name] = (data[col_name] - data[col_name].min()) / (data[col_name].max() - data[col_name].min())

In [37]:
drop_feat = ['label', 'uid', 'qid', 'dt']
drop_feat += ['gender_count','qid_enc_day_diff_iq_day_mean','qid_enc_day_diff_iq_day_median','uid_enc_day_score_mean',
              'uid_enc_day_score_median','uf_b1_day_diff_iq_hour_mean','uf_b2_day_diff_iq_hour_mean','uf_b3_day_diff_iq_hour_mean',
              'qid_enc_day_hour_diff_iq_day_mean','qid_enc_day_hour_diff_iq_day_median','qid_enc_day_hour_diff_iq_hour_mean',
              'qid_enc_day_hour_diff_iq_hour_median','uid_enc_day_hour_score_mean','uid_enc_day_hour_score_median',
              'uid_enc_qid_enc_score_mean','uid_enc_qid_enc_score_median','uid_enc_wk_score_mean','uid_enc_wk_score_median',
              'freq_wk_score_median','freq_wk_diff_iq_hour_mean','gender_wk_diff_iq_day_median','gender_wk_diff_iq_hour_mean',
              'uf_b1_wk_diff_iq_day_median','uf_b1_wk_diff_iq_hour_mean','uf_b2_wk_score_median','uf_b2_wk_diff_iq_day_median',
              'uf_b2_wk_diff_iq_hour_mean','uf_b3_wk_diff_iq_hour_mean','uf_b4_wk_score_median','uf_b4_wk_diff_iq_day_median',
              'uf_b4_wk_diff_iq_hour_mean','uf_b5_wk_diff_iq_day_median','uf_b5_wk_diff_iq_hour_mean','uf_c5_wk_diff_iq_hour_mean']
# drop_feat += ['u_is_rec_mean', 'u_reci_uncheer_mean', 'q_is_dest_sum', 'u_reci_uncheer_sum', 'u_is_rec_max', 
#              'u_is_dest_mean','q_reci_uncheer_mean', 'q_reci_uncheer_sum', 'u_is_dest_sum', 'q_is_dest_max',
#              'q_reci_uncheer_max', 'u_reci_tks_max', 'q_reci_mark_max','u_reci_dis_max', 'q_has_video_mean',
#              'q_reci_no_help_mean', 'count_u_topic', 'u_has_video_mean', 'q_reci_dis_sum', 'q_reci_mark_sum',
#              'q_reci_tks_sum','q_reci_tks_max','q_reci_dis_max','u_reci_mark_max','q_is_good_mean',
#              'q_reci_no_help_sum', 'q_reci_xxx_max', 'u_reci_xxx_max','u_reci_no_help_sum','u_reci_xxx_sum',
#               'u_is_good_mean','q_reci_no_help_max','u_has_img_max','u_is_good_sum','u_reci_no_help_max',
#               'u_has_video_sum','uf_b5','q_reci_xxx_sum','q_is_good_sum','q_has_img_max','q_has_video_sum',
#               'q_has_video_max','u_has_video_max','q_is_good_max','q_is_rec_max','u_is_good_max',
#               'q_is_dest_mean','u_reci_uncheer_max','uf_c5_count','u_is_dest_max','q_is_rec_mean',
#               'q_is_rec_sum','u_is_rec_sum', 'q_reci_xxx_mean','u_reci_xxx_mean','u_reci_comment_max',
#               'q_reci_comment_sum','u_reci_cheer_max','u_reci_dis_sum','u_reci_tks_sum','q_has_img_sum',
#               'q_reci_comment_max','q_reci_cheer_max','u_reci_no_help_mean','u_has_img_sum','u_reci_mark_sum']

feature_with_day = [x for x in data.columns if x not in drop_feat]
feature_cols = [x for x in data.columns if x not in drop_feat+['day']]
# feature_cols

In [51]:
logging.info("feature size %s", len(feature_cols))

X_train_all = data.iloc[:len_train][feature_with_day]
y_train_all = data.iloc[:len_train]['label']
X_test = data.iloc[len_train:]
assert len(X_test) == sub_size

# fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
#     break

# X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
#                                  y_train_all.iloc[train_idx], \
#                                  y_train_all.iloc[val_idx]
# del X_train_all

X_train = X_train_all.loc[X_train_all['day']<3867, feature_cols]
X_val = X_train_all.loc[X_train_all['day']==3867, feature_cols]
y_train = y_train_all[X_train_all['day']<3867]
y_val = y_train_all[X_train_all['day']==3867]
del X_train_all

logging.info("train shape %s, val shape %s, test shape %s", X_train.shape, X_val.shape, X_test.shape)


[2019-12-14 17:00:05,429] INFO in <ipython-input-51-a2460efdb538>: feature size 603
[2019-12-14 19:09:53,287] INFO in <ipython-input-51-a2460efdb538>: train shape (9141216, 603), val shape (347946, 603), test shape (1141683, 621)


In [52]:
X_train.columns

Index(['count_u_topic', 'dayscore_max', 'dayscore_mean', 'dayscore_median',
       'dayscore_min', 'dayscore_std', 'dayuf_b1_max', 'dayuf_b1_mean',
       'dayuf_b1_median', 'dayuf_b1_min',
       ...
       'uf_c4_day_hour_diff_iq_day_mean', 'uf_c4_day_hour_diff_iq_day_median',
       'uf_c4_day_hour_diff_iq_hour_mean',
       'uf_c4_day_hour_diff_iq_hour_median', 'uf_c5_day_hour_score_mean',
       'uf_c5_day_hour_score_median', 'uf_c5_day_hour_diff_iq_day_mean',
       'uf_c5_day_hour_diff_iq_day_median', 'uf_c5_day_hour_diff_iq_hour_mean',
       'uf_c5_day_hour_diff_iq_hour_median'],
      dtype='object', length=603)

In [ ]:
model_lgb = LGBMClassifier(n_estimators=2000, num_leaves=512, n_jobs=-1, objective='binary', learning_rate=0.01,
                           seed=1000, silent=True, max_bin=425, subsample_for_bin=50000, min_split_gain=0,max_depth=-1,
                          min_child_weight=1, min_child_samples=10, subsample=0.8, subsample_freq=1, is_unbalance=True, 
                          colsample_bytree=0.5, reg_alpha=2.99, reg_lambda=5)
# model_lgb = LGBMClassifier(n_estimators=2000, n_jobs=-1, objective='binary', seed=1000, silent=True)
model_lgb.fit(X_train, y_train,         
              eval_metric=['auc', 'logloss'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)

[1]	valid_0's auc: 0.818013	valid_0's binary_logloss: 0.431552
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.827079	valid_0's binary_logloss: 0.429416
[3]	valid_0's auc: 0.83262	valid_0's binary_logloss: 0.427433
[4]	valid_0's auc: 0.835924	valid_0's binary_logloss: 0.425388
[5]	valid_0's auc: 0.834422	valid_0's binary_logloss: 0.423747
[6]	valid_0's auc: 0.835238	valid_0's binary_logloss: 0.422019
[7]	valid_0's auc: 0.834491	valid_0's binary_logloss: 0.420529
[8]	valid_0's auc: 0.834702	valid_0's binary_logloss: 0.419016
[9]	valid_0's auc: 0.834546	valid_0's binary_logloss: 0.417786
[10]	valid_0's auc: 0.835254	valid_0's binary_logloss: 0.416388
[11]	valid_0's auc: 0.836156	valid_0's binary_logloss: 0.414968
[12]	valid_0's auc: 0.836509	valid_0's binary_logloss: 0.413777
[13]	valid_0's auc: 0.837097	valid_0's binary_logloss: 0.412456
[14]	valid_0's auc: 0.837282	valid_0's binary_logloss: 0.411317
[15]	valid_0's auc: 0.837593	valid_0's binary_logloss

In [54]:
pickle.dump(model_lgb, open('./model/model.pkl', 'wb'))

In [55]:
sub['label'] = model_lgb.predict_proba(X_test[feature_cols])[:, 1]
sub.to_csv('./result/2000.txt', index=None, header=None, sep='\t')

In [43]:
fi = pd.DataFrame({'feature': feature_cols, 'imp': model_lgb.feature_importances_})
fi['rate'] = fi['imp'] / fi['imp'].sum()
fi_sorted = fi.sort_values(by='rate', ascending=False)

In [44]:
fi_sorted.to_csv('./fi/fi.txt', sep='\t')

In [45]:
# import pickle
# pickle.dump(fi_sorted, open('./feature_importance.pkl', 'wb'))

In [46]:
fi_sorted[:60]

,feature,imp,rate
237,score,3216,0.012923
369,uid_history_label_mean_2,2665,0.010709
375,uid_hour_std,2301,0.009247
384,uid_enc_count,2143,0.008612
411,qid_enc_diff_iq_hour_count,1953,0.007848
372,uid_hour_mean,1844,0.007410
367,uid_history_inv_count_2,1788,0.007185
265,u_inv_kfold_mean,1662,0.006679
381,uid_week_std,1459,0.005863
255,sum_sim_norm_eucl,1361,0.005469


In [47]:
fi_sorted[-60:]

,feature,imp,rate
51,dayuf_c5_max,9,0.000036
303,uf_b4_max,7,0.000028
523,uf_c5_day_diff_iq_hour_mean,7,0.000028
483,uf_b5_day_diff_iq_hour_mean,6,0.000024
193,qid_wk_uf_b3_max,6,0.000024
21,dayuf_b4_max,5,0.000020
26,dayuf_b5_max,5,0.000020
198,qid_wk_uf_b4_max,5,0.000020
228,qid_wk_uf_c5_max,5,0.000020
188,qid_wk_uf_b2_max,5,0.000020


In [50]:
list(fi_sorted[fi_sorted['rate']==0]['feature'])

['uf_b4_min',
 'dayuf_b4_min',
 'dayuf_b4_median',
 'uf_c5_count',
 'dayuf_b3_min',
 'dayuf_b3_median',
 'qid_wk_uf_b4_median',
 'dayuf_b5_median',
 'dayuf_b2_median',
 'uf_b5_median',
 'dayuf_b1_min',
 'uf_b5_min',
 'wk',
 'uf_c5_min',
 'dayuf_b2_min',
 'dayuf_b5_min',
 'qid_wk_uf_b5_median',
 'uf_b3_median',
 'qid_enc_day_score_mean',
 'dayuf_c5_min',
 'qid_enc_day_score_median',
 'uf_b2_min',
 'uf_b1_min',
 'qid_wk_uf_b1_min',
 'qid_wk_uf_c5_min',
 'qid_wk_uf_b2_median',
 'qid_wk_uf_b2_min',
 'qid_wk_uf_b3_median',
 'qid_wk_uf_b3_min',
 'qid_wk_uf_b5_min',
 'qid_wk_uf_b4_min']